
# Context and Starting Point

The purpose of this work is to build a scalable pipeline that simplifies multilingual publishing across SRG SSR entities.

Within SRG SSR, several initiatives already address related use cases. This work builds on these existing foundations.

One key initiative is the **Publication Data Platform (PDP)**.  
The PDP aggregates all published articles from SRG SSR and its business units:

- Swissinfo  
- SRF  
- RTS  
- RSI  
- RTR  

All published content is consolidated into a centralized Kafka feed.

- [View Kafka topic data (AKHQ)](https://akhq.pdp.production.admin.srgssr.ch/ui/strimzi/topic/articles-v2/data?sort=NEWEST&partition=All)

---

# Infrastructure

A Databricks infrastructure is in place and operated within SRG SSR.

Workspace:  
https://adb-4119964566130471.11.azuredatabricks.net/

The Kafka feed is ingested into Databricks and available as the Delta table:

`udp_prd_modeled.pdp.articles_v2`

This table represents the **Bronze layer**, meaning the raw modeled Kafka data.

However, the structure still reflects the original Kafka schema and is therefore:

- Nested (JSON structure)  
- Not normalized  
- Not directly suited for analytics or downstream automation  

The data represents raw publication events and requires structural transformation before it can be used in analytical or AI-driven workflows.

Access to this infrastructure and its datasets is available only to entitled SRG staff.

---

# Purpose of this Notebook (Silver Layer)

This notebook represents the **Silver layer** of the pipeline.

Its goal is to:

1. Read the raw PDP article data from `udp_prd_modeled.pdp.articles_v2`.  
2. Explode nested JSON structures.  
3. Separate complex hierarchical structures into structured tables.  
4. Persist relationally accessible Delta tables for downstream processing.  

At this stage:

- No AI logic is implemented.  
- No content generation is performed.  
- Only structural normalization is applied.  

The result is a clean relational foundation for further transformations in Gold or AI application layers.

---

# Processing Steps

1. Read article data from `udp_prd_modeled.pdp.articles_v2`.  
2. Flatten and explode nested fields (e.g., titles, resources, contributors).  
3. Create structured Spark DataFrames.  
4. Persist the transformed data as Delta tables.  
5. Prepare relational access patterns for downstream usage.  

---

# Internal Infrastructure Notice

This notebook is developed and executed within the secured SRG SSR Databricks environment.

- The dataset may contain confidential publication data.  
- Access is restricted to entitled SRG staff.  
- Confidential datasets cannot be exported.  
- Code execution is limited to the Databricks (Spark) environment.  
- The infrastructure is not accessible to external users.  

This ensures compliance with SRG governance, data protection, and infrastructure policies.

---

# Note on AI Assistance

Parts of the transformation logic in this notebook were developed with AI assistance to accelerate structural exploration and flattening of complex nested JSON schemas.


🔧 1) Setup

In [0]:
# Databricks | PySpark
from pyspark.sql import functions as F

CATALOG = "swi_audience_prd"
SCHEMA  = "pdp_articles_v2_silver"
SRC     = "udp_prd_atomic.pdp.articles_v2"   # Bronze/Atomic-Quelle

spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}")

df = spark.table(SRC)

def save(df, table_name, mode="overwrite"):
    full = f"{CATALOG}.{SCHEMA}.{table_name}"
    (df.write
       .option("overwriteSchema", "true")
       .mode(mode)
       .saveAsTable(full))
    print(f"✅ wrote {full} ({df.count()} rows)")


📘 2) Core-Tabelle

In [0]:
core = (
    df.select(
        F.col("value.id").alias("article_id"),
        F.col("value.publisher").alias("publisher"),
        F.col("value.provenance").alias("provenance"),
        F.col("value.releaseDate").alias("releaseDate"),
        F.col("value.modificationDate").alias("modificationDate"),
        F.col("topic").alias("topic"),  # Top-Level
        # ⚠️ Ohne .items, da Struct:
        F.col("value.kicker.content").alias("kicker_default"),
        F.col("value.lead.content").alias("lead_default"),
        F.col("partition").alias("source_partition"),
        F.col("offset").alias("source_offset"),
        F.col("timestamp").alias("source_timestamp"),
        F.col("atomic_ts").alias("atomic_ts")
    )
    .dropDuplicates(["article_id"])
)
save(core, "core")

2) Helper: „items“-Erkennung + normalisierte Auswahl

In [0]:
from pyspark.sql.types import StructType, ArrayType

def has_items(value_schema: StructType, field_name: str) -> bool:
    f = next((x for x in value_schema.fields if x.name == field_name), None)
    if not f or not isinstance(f.dataType, StructType):
        return False
    return any(ch.name == "items" and isinstance(ch.dataType, ArrayType) for ch in f.dataType.fields)

value_schema = next(f for f in df.schema.fields if f.name == "value").dataType
TITLE_HAS_ITEMS  = has_items(value_schema, "title")
LEAD_HAS_ITEMS   = has_items(value_schema, "lead")
KICKER_HAS_ITEMS = has_items(value_schema, "kicker")

def write_lang_items(field_name: str, out_table: str, HAS_ITEMS: bool):
    if HAS_ITEMS:
        out = (
            df.select(
                F.col("value.id").alias("article_id"),
                F.posexplode_outer(F.col(f"value.{field_name}.items")).alias("pos","it")
            )
            .select(
                "article_id","pos",
                F.col("it.language").alias("language"),
                F.col("it.content").alias("content"),
                F.col("it.formatted.content").alias("formatted_content"),
                F.col("it.formatted.contentType").alias("contentType")
            )
        )
    else:
        # Einzelner Struct -> als ein „Item“ mit pos=0 ausgeben
        out = (
            df.where(F.col(f"value.{field_name}").isNotNull())
              .select(
                  F.col("value.id").alias("article_id"),
                  F.lit(0).alias("pos"),
                  F.col(f"value.{field_name}.language").alias("language"),
                  F.col(f"value.{field_name}.content").alias("content"),
                  F.col(f"value.{field_name}.formatted.content").alias("formatted_content"),
                  F.col(f"value.{field_name}.formatted.contentType").alias("contentType"),
              )
        )
    save(out, out_table)

write_lang_items("title",  "title_items",  TITLE_HAS_ITEMS)
write_lang_items("lead",   "lead_items",   LEAD_HAS_ITEMS)
write_lang_items("kicker", "kicker_items", KICKER_HAS_ITEMS)


🔧 Helper-Funktionen (robuste Array-Handhabung)

In [0]:
from pyspark.sql.types import StructType, ArrayType, StringType

value_schema = next(f for f in df.schema.fields if f.name == "value").dataType

def is_array(path: str) -> bool:
    """Check if a given path (relative to 'value') is an array."""
    # simple path walker: value.a.b.c
    cur = value_schema
    for name in path.split("."):
        if isinstance(cur, StructType):
            f = next((x for x in cur.fields if x.name == name), None)
            if not f: 
                return False
            cur = f.dataType
        elif isinstance(cur, ArrayType):
            cur = cur.elementType
        else:
            return False
    return isinstance(cur, ArrayType)

def has_items_struct(parent: str) -> bool:
    """Check if a struct has an 'items' Array child (e.g. title.items)."""
    f = next((x for x in value_schema.fields if x.name == parent), None)
    if not f or not isinstance(f.dataType, StructType):
        return False
    items = next((x for x in f.dataType.fields if x.name == "items"), None)
    return items is not None and isinstance(items.dataType, ArrayType)


🧩 A) Einfache Arrays: keywords, relatedArticles, existsAs, relatedEditorialObjects

In [0]:
from pyspark.sql import functions as F

def array_simple_strings(field: str, out_table: str, out_col: str):
    # Fälle:
    # 1) value.field.items[] existiert (Schema-Doku-Variante)
    # 2) value.field[] ist direkt ein Array
    # 3) value.field ist einzelner String
    if has_items_struct(field):
        out = (
            df.select(
                F.col("value.id").alias("article_id"),
                F.posexplode_outer(F.col(f"value.{field}.items")).alias("pos", out_col)
            )
        )
    elif is_array(field):
        out = (
            df.select(
                F.col("value.id").alias("article_id"),
                F.posexplode_outer(F.col(f"value.{field}")).alias("pos", out_col)
            )
        )
    else:
        # Einzelwert → pos=0
        out = (
            df.where(F.col(f"value.{field}").isNotNull())
              .select(
                  F.col("value.id").alias("article_id"),
                  F.lit(0).alias("pos"),
                  F.col(f"value.{field}").alias(out_col)
              )
        )
    save(out, out_table)

array_simple_strings("keywords",                "keywords",                    "keyword")
array_simple_strings("relatedArticles",         "related_articles",            "related_id")
array_simple_strings("existsAs",                "exists_as",                   "exists_as")
array_simple_strings("relatedEditorialObjects", "related_editorial_objects",   "editorial_id")


🧩 B) content.text (Array oder einzelner String)

In [0]:
def content_text_table():
    # Doku sagt: content.text.items; real kann auch string / array sein
    base_path = "content.text"
    if has_items_struct("content"):
        # Sonderfall: content hat items? (unwahrscheinlich hier) – fallback:
        out = (
            df.select(
                F.col("value.id").alias("article_id"),
                F.posexplode_outer(F.col("value.content.text.items")).alias("pos","text")
            )
        )
    elif is_array(base_path):
        out = (
            df.select(
                F.col("value.id").alias("article_id"),
                F.posexplode_outer(F.col("value.content.text")).alias("pos","text")
            )
        )
    else:
        out = (
            df.where(F.col("value.content.text").isNotNull())
              .select(
                  F.col("value.id").alias("article_id"),
                  F.lit(0).alias("pos"),
                  F.col("value.content.text").alias("text")
              )
        )
    save(out, "content_text")

content_text_table()


🧩 C) identifiers (Array von Structs)

In [0]:
# IDENTIFIERS – robust (items[] | direktes Array | einzelner Struct)
if has_items_struct("identifiers"):
    identifiers = (
        df.select(
            F.col("value.id").alias("article_id"),
            F.posexplode_outer("value.identifiers.items").alias("pos","i")
        ).select("article_id","pos",
                 F.col("i.type").alias("type"),
                 F.col("i.value").alias("value"))
    )
elif is_array("identifiers"):
    identifiers = (
        df.select(
            F.col("value.id").alias("article_id"),
            F.posexplode_outer("value.identifiers").alias("pos","i")
        ).select("article_id","pos",
                 F.col("i.type").alias("type"),
                 F.col("i.value").alias("value"))
    )
else:
    # einzelner Struct (selten, aber sicher)
    identifiers = (
        df.where(F.col("value.identifiers").isNotNull())
          .select(
              F.col("value.id").alias("article_id"),
              F.lit(0).alias("pos"),
              F.col("value.identifiers.type").alias("type"),
              F.col("value.identifiers.value").alias("value")
          )
    )

save(identifiers, "identifiers")


d) genres (Array von Structs – ohne .items unter name/description)

In [0]:
# GENRES – robust: genres.items  ODER  genres[]  ODER einzelner Struct
from pyspark.sql import functions as F

if has_items_struct("genres"):
    genres_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.genres.items").alias("pos","g")
    )
elif is_array("genres"):
    genres_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.genres").alias("pos","g")
    )
else:
    genres_src = (
        df.where(F.col("value.genres").isNotNull())
          .select(
              F.col("value.id").alias("article_id"),
              F.lit(0).alias("pos"),
              F.col("value.genres").alias("g")
          )
    )

genres = genres_src.select(
    "article_id","pos",
    F.col("g.name.content").alias("name_content"),
    F.col("g.name.language").alias("name_language"),
    F.col("g.name.formatted.content").alias("name_formatted_content"),
    F.col("g.name.formatted.contentType").alias("name_contentType"),
    F.col("g.description.content").alias("description_content"),
    F.col("g.description.language").alias("description_language"),
    F.col("g.description.formatted.content").alias("description_formatted_content"),
    F.col("g.description.formatted.contentType").alias("description_contentType"),
    F.col("g.sourceId").alias("sourceId")
)
save(genres, "genres")


e) contributors (Base + Occupations, robust)

In [0]:
# CONTRIBUTORS BASE – contributors.items ODER contributors[] ODER einzelner Struct
if has_items_struct("contributors"):
    contrib_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.contributors.items").alias("pos","c")
    )
elif is_array("contributors"):
    contrib_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.contributors").alias("pos","c")
    )
else:
    contrib_src = (
        df.where(F.col("value.contributors").isNotNull())
          .select(
              F.col("value.id").alias("article_id"),
              F.lit(0).alias("pos"),
              F.col("value.contributors").alias("c")
          )
    )

contributors_base = contrib_src.select(
    "article_id","pos",
    F.col("c.name").alias("contrib_name"),
    F.col("c.agent.person.name").alias("person_name"),
    F.col("c.agent.team.name").alias("team_name"),
    F.col("c.agent.department.name").alias("department_name")
)
save(contributors_base, "contributors_base")

# OCCUPATIONS – c.agent.person.occupations.items ODER occupations[] ODER einzelner Struct
# Wir hängen an contrib_src an und explodieren (falls vorhanden)
from pyspark.sql.types import ArrayType, StructType

def occ_is_array():
    # prüft: value.contributors[*].agent.person.occupations ist Array?
    return (is_array("contributors.items.agent.person.occupations") or
            is_array("contributors.agent.person.occupations"))

def occ_has_items():
    # prüft: ...occupations.items existiert?
    return (has_items_struct("contributors") and
            is_array("contributors.items.agent.person.occupations.items"))

if occ_has_items():
    occ = (
        contrib_src
        .select("article_id","pos","c",
                F.posexplode_outer("c.agent.person.occupations.items").alias("occ_pos","occ"))
    )
elif occ_is_array():
    occ = (
        contrib_src
        .select("article_id","pos","c",
                F.posexplode_outer("c.agent.person.occupations").alias("occ_pos","occ"))
    )
else:
    # einzelner Struct → pos=0, aber nur wenn vorhanden
    occ = (
        contrib_src
        .where(F.col("c.agent.person.occupations").isNotNull())
        .select("article_id","pos",
                F.lit(0).alias("occ_pos"),
                F.col("c.agent.person.occupations").alias("occ"))
    )

contributors_occupations = occ.select(
    "article_id",
    "pos",
    F.col("occ_pos").alias("occupation_pos"),
    # name als Struct mit optional formatted
    F.col("occ.name.language").alias("occupation_language"),
    F.col("occ.name.content").alias("occupation_content"),
    F.col("occ.name.formatted.content").alias("occupation_formatted_content"),
    F.col("occ.name.formatted.contentType").alias("occupation_contentType")
)
save(contributors_occupations, "contributors_occupations")


f) resources (Pictures / Documents / Links – ohne .items in name/description/altText)

In [0]:
# --- RESOURCES (robust, ohne identifiers im Documents-Table) ---
from pyspark.sql import functions as F

# Quelle für resources: items[] | [] | einzelner Struct
if has_items_struct("resources"):
    res_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.resources.items").alias("pos","r")
    )
elif is_array("resources"):
    res_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.resources").alias("pos","r")
    )
else:
    res_src = (
        df.where(F.col("value.resources").isNotNull())
          .select(
              F.col("value.id").alias("article_id"),
              F.lit(0).alias("pos"),
              F.col("value.resources").alias("r")
          )
    )

# PICTURES
pictures = (
    res_src.where(F.col("r.picture").isNotNull())
    .select(
        "article_id","pos",
        F.col("r.picture.name.language").alias("name_language"),
        F.col("r.picture.name.content").alias("name_content"),
        F.col("r.picture.description.language").alias("description_language"),
        F.col("r.picture.description.content").alias("description_content"),
        F.col("r.picture.locator.url").alias("url"),
        F.col("r.picture.locator.urn").alias("urn"),
        F.col("r.picture.width").alias("width"),
        F.col("r.picture.height").alias("height"),
        F.col("r.picture.copyright").alias("copyright"),
        F.col("r.picture.altText.language").alias("alt_language"),
        F.col("r.picture.altText.content").alias("alt_content"),
        F.col("r.picture.usage.sourceValue").alias("usage_sourceValue")
    )
)
save(pictures, "resources_pictures")

# DOCUMENTS  (ohne identifiers; wird in separater Tabelle behandelt)
documents = (
    res_src.where(F.col("r.document").isNotNull())
    .select(
        "article_id","pos",
        F.col("r.document.name.language").alias("name_language"),
        F.col("r.document.name.content").alias("name_content"),
        F.col("r.document.description.language").alias("description_language"),
        F.col("r.document.description.content").alias("description_content"),
        F.col("r.document.locator.url").alias("url"),
        F.col("r.document.locator.urn").alias("urn"),
        F.col("r.document.productionDate").alias("productionDate"),
        F.col("r.document.modificationDate").alias("modificationDate")
    )
)
save(documents, "resources_documents")

# LINKS
links = (
    res_src.where(F.col("r.link").isNotNull())
    .select(
        "article_id","pos",
        F.col("r.link.name").alias("name"),
        F.col("r.link.locator.url").alias("url"),
        F.col("r.link.locator.urn").alias("urn")
    )
)
save(links, "resources_links")


g) accessConditions (+ Geo-Whitelist – ohne .description.items)

In [0]:
# ACCESS CONDITIONS – accessConditions.items  ODER accessConditions[]  ODER einzelner Struct
if has_items_struct("accessConditions"):
    ac_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.accessConditions.items").alias("pos","a")
    )
elif is_array("accessConditions"):
    ac_src = df.select(
        F.col("value.id").alias("article_id"),
        F.posexplode_outer("value.accessConditions").alias("pos","a")
    )
else:
    ac_src = (
        df.where(F.col("value.accessConditions").isNotNull())
          .select(
              F.col("value.id").alias("article_id"),
              F.lit(0).alias("pos"),
              F.col("value.accessConditions").alias("a")
          )
    )

# Geo-Whitelist: a.geoBlockingWhitelist.items  ODER  a.geoBlockingWhitelist[]  ODER einzelner Struct
geo_items = (
    ac_src
    .select(
        "article_id","pos","a",
        F.posexplode_outer("a.geoBlockingWhitelist.items").alias("geo_pos","geo")
    )
    if (is_array("accessConditions.items.geoBlockingWhitelist.items")
        or is_array("accessConditions.geoBlockingWhitelist.items"))
    else (
        ac_src.select(
            "article_id","pos","a",
            F.posexplode_outer("a.geoBlockingWhitelist").alias("geo_pos","geo")
        ) if (
            is_array("accessConditions.items.geoBlockingWhitelist")
            or is_array("accessConditions.geoBlockingWhitelist")
        )
        else ac_src.where(F.col("a.geoBlockingWhitelist").isNotNull())
                   .select("article_id","pos","a",
                           F.lit(0).alias("geo_pos"),
                           F.col("a.geoBlockingWhitelist").alias("geo"))
    )
)

access_conditions = geo_items.select(
    "article_id","pos",
    F.col("a.name").alias("name"),
    F.col("a.description.content").alias("description_content"),
    F.col("a.availableFrom").alias("availableFrom"),
    F.col("a.availableTo").alias("availableTo"),
    F.col("a.blockReason").alias("blockReason"),
    F.col("a.embeddingDisabled").alias("embeddingDisabled"),
    F.col("geo_pos").alias("geoWhitelist_pos"),
    F.col("geo.sourceValue").alias("geoWhitelist_value")
)
save(access_conditions, "access_conditions")
